In [101]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [102]:
# Configuration de l'affichage pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [103]:
# Liste des villes
cities = ["Paris", "Lyon", "Marseille", "Toulouse", "Bordeaux", "Nantes", "Nice"]

In [104]:
def clean_price(price_text):
    try:
        # Garder uniquement les chiffres, points et virgules
        price = price_text.replace(' ', '').strip()
        if '€' in price:
            price = price.replace('€', '').strip()
        price = price.replace(',', '.')
        
        # Convertir en float et formater avec le symbole €
        if price and price.replace('.', '').isdigit():
            value = float(price)
            return f"{value:.2f} €"
        return price_text.strip()
    except:
        return price_text.strip()



In [105]:
def get_cost_of_living(city):
    try:
        url = f"https://www.numbeo.com/cost-of-living/in/{city}?displayCurrency=EUR"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        data = {}
        data['City'] = city
        
        # Trouver toutes les tables de la page
        tables = soup.find_all('table', class_='data_wide_table')
        
        for table in tables:
            for row in table.find_all('tr'):
                cells = row.find_all('td')
                if len(cells) >= 2:
                    item_name = cells[0].text.strip()
                    price_text = cells[1].text.strip()
                    
                    # Ne pas ajouter le symbole € pour le taux d'intérêt
                    if "Mortgage Interest Rate" in item_name:
                        price = clean_price(price_text).replace(' €', '')
                    else:
                        price = clean_price(price_text)
                        
                    if item_name and price_text:
                        data[item_name] = price
        
        return data
        
    except Exception as e:
        print(f"Erreur pour {city}: {str(e)}")
        return None

In [106]:
# Collecte des données avec délai entre les requêtes
all_data = []
for city in cities:
    print(f"Récupération des données pour {city}...")
    data = get_cost_of_living(city)
    if data:
        all_data.append(data)
    time.sleep(2)  # Attendre 2 secondes entre chaque requête


Récupération des données pour Paris...
Récupération des données pour Lyon...
Récupération des données pour Marseille...
Récupération des données pour Toulouse...
Récupération des données pour Bordeaux...
Récupération des données pour Nantes...
Récupération des données pour Nice...


In [107]:
# Création du DataFrame
df = pd.DataFrame(all_data)

# Définir 'City' comme index avant la transposition
df.set_index('City', inplace=True)

# Transposer le DataFrame
df_transposed = df.transpose()

# Sauvegarde des données
df_transposed.to_csv("cost_of_living_cleaned.csv")
print("Données sauvegardées dans 'cost_of_living_cleaned.csv'")

# Affichage des données
print("\nAperçu complet des données :")
display(df_transposed)

Données sauvegardées dans 'cost_of_living_cleaned.csv'

Aperçu complet des données :


City,Paris,Lyon,Marseille,Toulouse,Bordeaux,Nantes,Nice
"Meal, Inexpensive Restaurant",15.00 €,15.00 €,15.00 €,14.50 €,17.50 €,15.00 €,20.00 €
"Meal for 2 People, Mid-range Restaurant, Three-course",70.00 €,70.00 €,62.50 €,60.00 €,60.00 €,55.00 €,80.00 €
McMeal at McDonalds (or Equivalent Combo Meal),12.00 €,11.00 €,10.00 €,11.00 €,10.00 €,10.00 €,10.50 €
Domestic Beer (0.5 liter draught),7.00 €,7.00 €,6.85 €,6.75 €,7.50 €,7.00 €,7.00 €
Imported Beer (0.33 liter bottle),3.43 €,2.45 €,2.45 €,3.00 €,3.75 €,2.83 €,2.20 €
Cappuccino (regular),3.91 €,3.41 €,3.14 €,2.88 €,3.50 €,3.06 €,3.09 €
Coke/Pepsi (0.33 liter bottle),3.88 €,2.87 €,2.60 €,2.42 €,2.41 €,2.30 €,3.04 €
Water (0.33 liter bottle),2.92 €,1.64 €,1.61 €,1.56 €,1.34 €,1.30 €,2.29 €
"Milk (regular), (1 liter)",1.33 €,1.35 €,1.19 €,1.27 €,1.08 €,1.01 €,1.41 €
Loaf of Fresh White Bread (500g),1.95 €,2.38 €,2.12 €,2.00 €,1.85 €,1.85 €,1.58 €
